In [1]:
println(s"Current spark version is ${spark.version}")

Intitializing Scala interpreter ...

Spark Web UI available at http://e4923597481e:4041
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1598282466489)
SparkSession available as 'spark'


Current spark version is 3.0.0


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath: String = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment: org.apache.spark.sql.DataFrame = [label: int, tweet: string]


In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val lr = new RandomForestClassifier()
    .setNumTrees(12)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))


import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_f42f7015fffe
hashingTF: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_7fca34967727, binary=false, numFeatures=1000
lr: org.apache.spark.ml.classification.RandomForestClassifier = rfc_81433e8aec56
pipeline: org.apache.spark.ml.Pipeline = pipeline_f0c1e0306121


In [4]:
val model = pipeline.fit(raw_sentiment)

model: org.apache.spark.ml.PipelineModel = pipeline_f0c1e0306121


In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel: org.apache.spark.ml.PipelineModel = pipeline_f0c1e0306121


In [7]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[10,21,81,1...|[5.66169530055051...|[0.47180794171254...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[121,193,20...|[5.84912325609812...|[0.48742693800817...|       1.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[17,185,188...|[6.24375073288545...|[0.52031256107378...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[191,330,44...|[5.8483216888698,...|[0.48736014073914...|       1.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[32,162,166...|[6.46087170814505...|[0.53840597567875...|       0.0|
|    0|@

predictionsDF: org.apache.spark.sql.DataFrame = [label: int, tweet: string ... 5 more fields]


In [8]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [9]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

import org.apache.spark.sql.functions._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4861/0x00000008419ce040@36ba5560,DoubleType,List(Some(class[value[0]: vector])),None,false,true)


In [10]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
|  0.528192058287457|
| 0.5125730619918231|
|0.47968743892621185|
| 0.5126398592608501|
|0.46159402432124574|
| 0.5031871374202178|
| 0.5018048170965372|
| 0.5331736791865055|
| 0.5132386596607745|
|0.48353230593978713|
| 0.5160602968283927|
|0.48558346880733333|
|0.49317548860880606|
|0.48704913476027684|
|0.46019125504361696|
| 0.4713130264858678|
| 0.4821574583200135|
| 0.5202318790623227|
| 0.5307978468610002|
| 0.5165603195757178|
+-------------------+
only showing top 20 rows



In [11]:
spark.stop()